## <span style="color:red">User Inputs:</span>
<span style="color:red">Filename <br>
Sample geometry</span>

In [ ]:
filename = "SQUIDdata-ppblsmo/ppblsmo05-MvsT-cool7T-20170124.rso.dat"
samplename = 'ppblsmo05'
width = 3. #mm
height = 5. #mm
thickness = 9. #nm

In [ ]:
#Imports, setting, etc.
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook
from scipy import interpolate
from scipy import optimize
import numpy.lib.recfunctions
import time

#function to read in data from a MultiVu file
def read(fname):
    #SEPARATE HEADER FROM DATA
    headerline = ''
    header = '' #header will be added to the beginning of any output files
    nline = 0
    inputfile = open(fname,'r')
    while headerline != "[Data]\n":
        headerline = inputfile.readline()
        nline += 1
        header = header + headerline
    inputfile.close()
    #READ IN DATA FILE, masked. Not all of them will be filled. The mask has values of True where data is missing
    arr = np.genfromtxt(fname, delimiter=',', skip_header=nline, names=True, usemask=True, dtype=None)
    yield arr
    yield header

Read in data in MultiVu format

In [ ]:
data, header = read(filename)
#output data columns
print header

preheader = '[Header - Processing]\n'
preheader += 'File generated at '+'[TODO:insert time here]'+' from file: '+filename+'\n'

data.dtype.names

## Checking the data to make sure it's good quality

In [ ]:
#check that we are varying the right variables over time
plt.figure()
plt.plot(data['Field_Oe'],label='Field')
plt.plot(data['Temperature_K'],label='Temp')
plt.legend()

In [ ]:
plt.figure()
plt.plot(data['Temperature_K'],data['Long_Moment_emu'])

In [ ]:
plt.figure()
plt.plot(data['Long_Reg_Fit'],'.')

Filter out curves with bad fits if necessary

In [ ]:
filterlevel = .9
filt = np.ma.where(data['Long_Reg_Fit'] < filterlevel)
#filter out values with bad fit values - this masks the whole row - useful for compressing these rows
data[filt] = np.ma.masked

Inspect raw data to see why fit failed

In [ ]:
rawfilename = filename.replace('SQUIDdata','Raw').replace('.dat','.raw')
rawdata, rawheader = read(rawfilename)

In [ ]:
size = rawdata.size/data.size
for index in filt[0]:
    plt.figure()
    plt.title('Raw Scan for H='+str(rawdata['Field_Oe'][index*size])+'Oe')
    plt.plot(rawdata['Position_cm'][index*size:(index+1)*size],rawdata['Long_Voltage'][index*size:(index+1)*size],label='Original')
    plt.plot(rawdata['Position_cm'][index*size:(index+1)*size],rawdata['Long_Average_Voltage'][index*size:(index+1)*size],label='Averaged')
    plt.plot(rawdata['Position_cm'][index*size:(index+1)*size],rawdata['Long_Reg_Fit'][index*size:(index+1)*size],label='Fit')
    plt.legend()

TODO: remove artifacts from raw data and fit, if possible

Supress masked data - since masked arrays do not work well with many numpy routines (such as interpolate)

In [ ]:
#preheader += 'Data with fits below '+str(filterlevel)+' were removed from the data due to issues with the raw measurement/fit.\n'

importantcolumns = ['Temperature_K','Field_Oe','Long_Moment_emu','Long_Reg_Fit']
thedata = data[importantcolumns]
colstoextract = ~np.ma.getmaskarray(thedata).view('bool').reshape(thedata.shape + (-1,))[:,0]
gooddata = thedata[colstoextract]
gooddata

## Subtract background
TODO: figure out how to subtract background - maybe this requires an MvsH loop? Or is the background too small in this case?

In [ ]:
plt.figure()
plt.plot(gooddata['Temperature_K'], gooddata['Long_Moment_emu'])

In [ ]:
volume = width/10. * height/10. * thickness/1.E7 #cm^3
moment = gooddata['Long_Moment_emu']/volume #emu/cc
preheader += 'Moment scaled to volume assuming the following dimensions:\n'
preheader += '  width: '+str(width)+' mm, height: '+str(height)+' mm, thickness: '+str(thickness)+' nm\n'

plt.figure()
plt.plot(gooddata['Temperature_K'], moment, 'b.-')
plt.title('MvsT')
plt.xlabel('Temperature (K)')
plt.ylabel('Moment (emu/cc)')
ax = plt.gca()
ax.tick_params(direction='in')

## Extract parameters from MvsT curve and plot

Saturation magnetization<Br>
TODO: it's not actually saturation, right? Also, what to do about maxima in the MvsT - implies frozen in disorder of some sort

Curie temperature<br>
Interpolate data in order to extract zero crossings<br>
<span style="color:red">Multiple interpolation/fitting routines available</span>

In [ ]:
x = range(int(gooddata['Temperature_K'][0]),int(gooddata['Temperature_K'][-1]),1) #values to plot interpolated/fit function to verify that it's a good representation of the data

#TODO: fix interpolation and value extractions so that it is monotonic, etc.
#e.g. https://stackoverflow.com/questions/17935779/constrained-spline-fit-using-scipy-in-python
#TODO: propagate errors from saturated moment to exchange bias, remanent, coercivity, etc.
interpolationtype = 1

if interpolationtype == 1:
    #The interpolator preserves monotonicity in the interpolation data and does not overshoot if the data is not smooth. The first derivatives are guaranteed to be continuous, but the second derivatives may jump at x_k
    func = interpolate.PchipInterpolator(np.ma.getdata(gooddata['Temperature_K']),np.ma.getdata(moment))
    y = func(list(x))
elif interpolationtype == 2:
    #interp1d - linear spline
    func = interpolate.interp1d(gooddata['Temperature_K'], moment, kind='slinear')
    y = func(list(x))
elif interpolationtype == 3:
    #spline with smoothing
    func = interpolate.splrep((gooddata['Temperature_K']),moment, k=3,s=12)
    y = interpolate.splev(x,func)
elif interpolationtype == 4:
    #Use only for precise data, as the fitted curve passes through the given points exactly. This routine is useful for plotting a pleasingly smooth curve through a few given points for purposes of plotting.
    func = interpolate.Akima1DInterpolator(np.ma.getdata(gooddata['Field_Oe']), np.ma.getdata(moment))
    y = func(list(x))

plt.figure()
plt.plot(x,y,'b')
plt.plot(gooddata['Temperature_K'],moment,'r.')

In [ ]:
preheader += 'Curve interpolated/smoothed using '
if interpolationtype == 1:
    preheader += 'PchipInterpolator (monotonic, no smoothing)'
elif interpolationtype == 2:
    preheader += 'interp1d (linear spline)'
elif interpolationtype == 3:
    preheader += 'splrep (cubic spline with 12-point smoothing)'
elif interpolationtype == 4:
    preheader += 'Akima1DInterpolator (visually smooth curve)'
preheader += ' to extract parameters:\n'

In [ ]:
#TODO: rewrite calculations using uncertainty package, so errors propogate correctly
def tomin(x):
    return -func(x)
saturation = optimize.minimize_scalar(tomin,bracket=[80,110])
print saturation
satmoment = func(saturation.x)
print('Saturation moment: '+str(satmoment)+' emu/cm^3')
preheader += 'Saturation moment:    '+str(satmoment)+' emu/cm^3\n'

In [ ]:
Tc = optimize.brentq(func, 330, 345) #where the signal switches from ferromagnetic to diamagnetic - Tc may actually be slightly above this? would depend on the background tbh
print('Curie temperature: '+str(Tc)+' K')
preheader += 'Curie Temperature:       '+str(Tc)+' K\n'

In [ ]:
remanentdown = funcdown(0)
remanentup = funcup(0)
remanent = (remanentdown - remanentup)/2.
print('Remanent Magnetization: '+str(remanent)+' emu/cc')
print remanentdown
print remanentup
print funcdown(Hexchangebias)
print funcup(Hexchangebias)
preheader += 'Remanent moment:      '+str(remanent)+' emu/cm^3\n'

In [ ]:
plt.figure()
plt.plot(gooddata['Temperature_K'], moment,'r.-',label='Sweep')
plt.plot(saturation.x,satmoment,'kx',label='Saturation magnetization')
plt.vlines(Tc,0,satmoment,linestyles='dotted',label='Curie Temperature')
plt.title('MvsT: '+samplename)
plt.xlabel('Temperature (K)')
plt.ylabel('Moment (emu/cc)')
ax = plt.gca()
plt.legend(loc=3)
ax.tick_params(direction='in')
print('Saturation moment: '+str(satmoment)+' emu/cm^3')
#print('Remanent moment: '+str(remanent)+' emu/cm^3')
print ('Curie Temperature: '+str(Tc)+' K')
#print ('Exchange bias field: '+str(Hexchangebias)+' Oe')

## Calculate expected moment
<span style="color:red">Doping, chemical composition, etc. needed</span>

In [ ]:
Ladoping = 2./3.
Mnvalence = Ladoping*3. + (1.-Ladoping)*2. -3.*2.
delectronsperuc = 5 - Mnvalence
#octahedral field splitting - 3 t2g then 2 eg bands above
if delectronsperuc <= 3:
    momperuc = delectronsperuc
elif delectronsperuc <= 6:
    momperuc = 6.-delectronsperuc
else:
    momperuc = delectronsperuc-6.
density = 6.5 #g/cm^3
numuc = density * 6.022E23 / (Ladoping*138.91 + (1.-Ladoping)*87.62 + 54.938 + 3.*15.999) #g/uc
print numuc
print momperuc
expmoment = momperuc * numuc * (2 * 9.271E-21) #2 bohr magneton per electron
print 'Expected Saturation Moment: '+str(expmoment)+' emu/cm^3'
preheader += 'Expected sat. moment: '+str(expmoment)+' emu/cm^3\n'

## Export data to external file for graphing later

In [ ]:
#columnNames = 'Field (Oe), Temperature (K), Subtracted Moment (emu/cm3), Long Reg Fit'
outfilename = filename[:-8]+'-subtracted'+filename[-8:]
outdata = np.lib.recfunctions.append_fields(gooddata, 'Subtracted_Moment_emu/cm3', data=moment)
outcols = ['Temperature_K','Field_Oe','Subtracted_Moment_emu/cm3','Long_Reg_Fit']
outcolLabels = ['Temperature (K)', 'Field (Oe)', 'Subtracted Moment (emu/cm^3)', 'Long Reg Fit']
outheader = preheader.replace('[TODO:insert time here]', time.strftime('%c',time.localtime()))+'\n'+header+','.join(outcolLabels)

np.savetxt(outfilename, outdata[outcols], delimiter=',', header=outheader)
print 'Saved to: '+outfilename